<a href="https://colab.research.google.com/github/strivedi2/Gun-Violence-in-United-States/blob/master/Exploratory_Data_Analysis_Team3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gun Violence in United States

---

Project Team 4 <br>
**Team Members**: Bharati Malik, Gaurav Hassija, Prachi Sharma, Shruti Trivedi, Vikita Nayak

## Introduction
This project is inspired by the [Vox article](https://www.vox.com/policy-and-politics/2017/10/2/16399418/us-gun-violence-statistics-maps-charts) on America's unique **Gun Violence** problem. The article provides 17 maps and charts that represent various aspects of Gun ownership in United States. <br> 


##**Background**

US has a lot of guns, and very loose or non-existent regulations relating to who may access firearms makes it unique in terms of gun violence among other developed nations.


##**Search for relevant data**

We were interested in exploring Gun ownership within each state and its relationship with suicide rates, mass shootings and officer involved shootings. After going through various sources of data, we were directed to [Gun Violence Archive](https://www.gunviolencearchive.org/). As the website states, it is an online archive of gun violence incidents collected from over 2,500  law enforcement, media, government and commercial sources daily in an effort to provide near-real time data about the results of gun violence. 

However, the data available for download on the website is limited by the number of rows and attributes that can  be exported as CSV. This led us to a larger and richer dataset on [Kaggle](https://www.kaggle.com/jameslko/gun-violence-data/downloads/gun-violence-data.zip/1). 

In [0]:
import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',200)
pd.set_option('display.max_colwidth', -1)

## Dataset 1: Gun Violence Dataset on Kaggle
We downloaded the gun violence dataset available on [Kaggle](https://www.kaggle.com/jameslko/gun-violence-data/downloads/gun-violence-data.zip/1). This dataset has been taken from the same website [Gun Violence Archive](https://www.gunviolencearchive.org) using web scraping with Python script (_We could not get access to the script_). 

The dataset has a record of more than **260,000 gun violence incidents** between **Jan 2013 - March 2018**. 

In addition to incident date, state, address and number of people injured and killed; the dataset also contains information about the **type of incident** (such as mass shootings, suicides, officer involved shootings etc.), **guns involved** and **participant information including age, gender** etc. which makes this a richer dataset when compared to reports available on  [Gun Violence Archive](https://www.gunviolencearchive.org).


### Preliminary Data Cleaning using MS Excel

In preliminary analysis of the dataset, following columns were deleted in the csv file before uploading on github as they were not relevant to our analysis.


*   Address, location description : We decided to retain State, City, Latitutde and longitude for each incident and  believe address is not pertinent to our analysis.

*   In addition URL columns directing to source of incident were also deleted.




Next, separate csv files were created for each year from 2013 - 2018 (to satisfy the 25MB file limit on github)and uploaded on [GitHub](https://github.com/strivedi2/Gun-Violence-in-United-States).  We read those files in the following lines of code.



In [0]:
guns_2013 = pd.read_csv('https://raw.githubusercontent.com/strivedi2/Gun-Violence-in-United-States/master/gun-violence-2013.csv')
guns_2014 = pd.read_csv('https://raw.githubusercontent.com/strivedi2/Gun-Violence-in-United-States/master/gun-violence-2014.csv')
guns_2015 = pd.read_csv('https://raw.githubusercontent.com/strivedi2/Gun-Violence-in-United-States/master/gun-violence-2015.csv')
guns_2016 = pd.read_csv('https://raw.githubusercontent.com/strivedi2/Gun-Violence-in-United-States/master/gun-violence-2016.csv')
guns_2017 = pd.read_csv('https://raw.githubusercontent.com/strivedi2/Gun-Violence-in-United-States/master/gun-violence-2017.csv')
guns_2018 = pd.read_csv('https://raw.githubusercontent.com/strivedi2/Gun-Violence-in-United-States/master/gun-violence-2018.csv')

We concatenated the above dataframes to create a single dataframe.

In [0]:
guns_df = guns_2013.append([guns_2014,guns_2015,guns_2016,guns_2017,guns_2018])

We then checked for total number of columns and rows and identify if there are any columns with very high number of null values.

In [4]:
guns_df.shape

(239677, 19)

In [5]:
# To check for null values in the dataset
guns_df.isna().sum()

incident_id                 0    
date                        0    
state                       0    
city_or_county              0    
n_killed                    0    
n_injured                   0    
congressional_district      11944
gun_stolen                  99498
gun_type                    99451
incident_characteristics    326  
latitude                    7923 
longitude                   7923 
n_guns_involved             99451
participant_age_group       42119
participant_gender          36362
participant_status          27626
participant_type            24863
state_house_district        38772
state_senate_district       32335
dtype: int64

### **Data Cleaning**

**Drop columns with high null values** <br>
We dropped columns where the null values were greater than 90,000 since we believe they will not be relevant to our analyses. <br>

In [0]:
guns_df.drop(columns =['gun_stolen','gun_type','n_guns_involved'], inplace = True)

At this stage, the dataframe has following columns

In [7]:
guns_df.columns

Index(['incident_id', 'date', 'state', 'city_or_county', 'n_killed',
       'n_injured', 'congressional_district', 'incident_characteristics',
       'latitude', 'longitude', 'participant_age_group', 'participant_gender',
       'participant_status', 'participant_type', 'state_house_district',
       'state_senate_district'],
      dtype='object')

In [8]:
guns_df.head(2)

,incident_id,date,state,city_or_county,n_killed,n_injured,congressional_district,incident_characteristics,latitude,longitude,participant_age_group,participant_gender,participant_status,participant_type,state_house_district,state_senate_district
0,461105,1/1/13,Pennsylvania,Mckeesport,0,4,14.0,"Shot - Wounded/Injured||Mass Shooting (4+ victims injured or killed excluding the subject/suspect/perpetrator, one location)||Possession (gun(s) found during commission of other crimes)||Possession of gun by felon or prohibited person",40.3467,-79.8559,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::Adult 18+||4::Adult 18+,0::Male||1::Male||3::Male||4::Female,0::Arrested||1::Injured||2::Injured||3::Injured||4::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4::Subject-Suspect,NaN,NaN
1,460726,1/1/13,California,Hawthorne,1,3,43.0,"Shot - Wounded/Injured||Shot - Dead (murder, accidental, suicide)||Mass Shooting (4+ victims injured or killed excluding the subject/suspect/perpetrator, one location)||Gang involvement",33.9090,-118.3330,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::Adult 18+,0::Male,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4::Subject-Suspect,62.0,35.0


We have created Date Fields from the date column to help us plot yearly, monthly or weekly gun violence trends.

In [0]:
# Creating Date fields 

guns_df['date'] = pd.to_datetime(guns_df['date'])
guns_df['year'] = guns_df['date'].dt.year
guns_df['month'] = guns_df['date'].dt.month
guns_df['monthday'] = guns_df['date'].dt.day
guns_df['weekday'] = guns_df['date'].dt.weekday

### **Understanding types of gun violence incidents**

We wanted to categorize each incident into - **Mass Shooting, Suicide and/ or Officer Involved Shooting.**

We checked for above keywords in the incident_characateristics column. Since these are not mutually exclusive incidents we created new columns to indicate 1 if it was a Mass Shooting and 0 if not. Same approach was followed for Suicide and/ or Officer Involved Shooting columns.

In [0]:
# New columns for Mass Shootings, Suicides, Officer Involved Shooting in original dataframe

guns_df['Mass_Shooting'] = guns_df['incident_characteristics'].str.contains("Mass Shooting") * 1.0

guns_df['Officer_Involved_Shooting'] = guns_df['incident_characteristics'].str.contains("Officer Involved Shooting") * 1.0

guns_df['Suicide'] = guns_df['incident_characteristics'].str.contains("Suicide") * 1.0


We also found a high number of incidents involving **Gun possession by felons and Accidental Shootings** and created new columns to record these characterstics. 

We believe these aspects are important for analysis across different states. States with stricter laws should ideally be not allowing guns to get into hands of felons. 

Also, we want to understand how many people are injured due to accidental shootings.

In [0]:
guns_df['Gun_Possession_felon'] = guns_df['incident_characteristics'].str.contains("Possession of gun by felon") * 1.0

guns_df['Accidental_Shootings'] = guns_df['incident_characteristics'].str.contains("Accidental Shooting") * 1.0

In [0]:
# Drop columns incident_characterstics
guns_df.drop(columns ='incident_characteristics', inplace = True)

In [13]:
guns_df.columns

Index(['incident_id', 'date', 'state', 'city_or_county', 'n_killed',
       'n_injured', 'congressional_district', 'latitude', 'longitude',
       'participant_age_group', 'participant_gender', 'participant_status',
       'participant_type', 'state_house_district', 'state_senate_district',
       'year', 'month', 'monthday', 'weekday', 'Mass_Shooting',
       'Officer_Involved_Shooting', 'Suicide', 'Gun_Possession_felon',
       'Accidental_Shootings'],
      dtype='object')

### Overall trend of gun violence incidents

In [14]:
# Plotting yearly trend of gun violence incidents in United States
alt.Chart(guns_df.groupby('year')['incident_id'].count().reset_index()).mark_line().encode(
    alt.X('year:O', title = 'Year'),
    alt.Y('incident_id', title = 'Total number of gun violence incidents')
).properties(
    title = 'Trend of gun violence incidents, Jan. 2013 - Mar. 2018',
    width=600).configure_axis(
    grid=False
)



Chart({
  config: Config({
    axis: AxisConfig({
      grid: False
    })
  }),
  data:    year  incident_id
  0  2013  278        
  1  2014  51854      
  2  2015  53579      
  3  2016  58763      
  4  2017  61401      
  5  2018  13802      ,
  encoding: FacetedEncoding({
    x: X({
      shorthand: 'year:O',
      title: 'Year'
    }),
    y: Y({
      shorthand: 'incident_id',
      title: 'Total number of gun violence incidents'
    })
  }),
  mark: 'line',
  title: 'Trend of gun violence incidents, Jan. 2013 - Mar. 2018',
  width: 600
})

**Number of mass shootings by state.**

In [15]:
alt.Chart(guns_df.groupby('state')['Mass_Shooting'].count().reset_index()).mark_bar().encode(
    alt.X('state', title = 'State', 
         sort=alt.EncodingSortField(
            field="Mass_Shooting",
            op="sum",
            order="descending"
        )),
    alt.Y('Mass_Shooting', title = 'Total number of mass shootings')
).properties(
    title='Number of mass shootings by state (2013-2018)',
    width=650,
    height=400
).configure_axis(
    grid=False
)

Chart({
  config: Config({
    axis: AxisConfig({
      grid: False
    })
  }),
  data:                    state  Mass_Shooting
  0   Alabama               5470         
  1   Alaska                1349         
  2   Arizona               2325         
  3   Arkansas              2841         
  4   California            16289        
  5   Colorado              3199         
  6   Connecticut           3064         
  7   Delaware              1685         
  8   District of Columbia  3191         
  9   Florida               15020        
  10  Georgia               8922         
  11  Hawaii                288          
  12  Idaho                 661          
  13  Illinois              17506        
  14  Indiana               5835         
  15  Iowa                  2511         
  16  Kansas                2145         
  17  Kentucky              4148         
  18  Louisiana             8083         
  19  Maine                 907          
  20  Maryland              5792         
  21  Massachusetts         5976         
  22  Michigan              6113         
  23  Minnesota             2405         
  24  Mississippi           3597         
  25  Missouri              6628         
  26  Montana               638          
  27  Nebraska              1651         
  28  Nevada                1948         
  29  New Hampshire         964          
  30  New Jersey            5383         
  31  New Mexico            1642         
  32  New York              9703         
  33  North Carolina        8732         
  34  North Dakota          572          
  35  Ohio                  10225        
  36  Oklahoma              3450         
  37  Oregon                2286         
  38  Pennsylvania          8925         
  39  Rhode Island          895          
  40  South Carolina        6933         
  41  South Dakota          543          
  42  Tennessee             7617         
  43  Texas                 13539        
  44  Utah                  1072         
  45  Vermont               472          
  46  Virginia              5932         
  47  Washington            3433         
  48  West Virginia         1575         
  49  Wisconsin             4778         
  50  Wyoming               493          ,
  encoding: FacetedEncoding({
    x: X({
      shorthand: 'state',
      sort: EncodingSortField({
        field: 'Mass_Shooting',
        op: 'sum',
        order: 'descending'
      }),
      title: 'State'
    }),
    y: Y({
      shorthand: 'Mass_Shooting',
      title: 'Total number of mass shootings'
    })
  }),
  height: 400,
  mark: 'bar',
  title: 'Number of mass shootings by state (2013-2018)',
  width: 650
})

### **Limitations in the Dataset**

One of the challenges encountered in our analysis were caused by how the gun violence incidents were recorded in the dataset. Each entry in the dataset identifies an incident. Hence essential columns like Age, Participant type(suspect or victim), Gender were recorded in group. 



Participant Gender  | Participant Type
--- | ---
0::Male\|1::Male\|2::Male\|3::Female | 0::Victim\|1::Victim\|2::Victim\|3::Subject-Suspect


The challenge was to map these columns with each other to derive insights. For instance to know the Age of a particular participant like Suspect of the incident required valid mapping. Each record had different number of participants which made the extraction of the values even difficult.

For analysing the profile of Victims and Suspects we followed approach taken in this  [Kaggle Kernel](https://www.kaggle.com/shivamb/deep-exploration-of-gun-violence-in-us).

In [0]:
## Function for converting the values in form of key, value pair
def get_user_mapping(txt):
    if txt == "NA":
        return {}
    mapping = {}
    for d in txt.split("||"):
        try:
            key = d.split("::")[0]
            val = d.split("::")[1]
            if key not in mapping:
                mapping[key] = val
        except:
            pass

    return mapping

In [0]:
# Creating mapped columns for participants age group, type and gender
guns_df['participant_type'] = guns_df['participant_type'].fillna("NA")
guns_df['participant_type_map'] = guns_df['participant_type'].apply(lambda x : get_user_mapping(x))
guns_df['participant_age_group'] = guns_df['participant_age_group'].fillna("NA")
guns_df['participant_age_map'] = guns_df['participant_age_group'].apply(lambda x : get_user_mapping(x))
guns_df['participant_gender'] = guns_df['participant_gender'].fillna("NA")
guns_df['participant_gender_map'] = guns_df['participant_gender'].apply(lambda x : get_user_mapping(x))

### **Understanding Gun Violence Suspects Profile**

We wanted to analyze age and gender of suspects and victims of gun violence to identify any significant trends/insights.


For analysing the profile of Victims and Suspects we followed approach taken in this  [Kaggle Kernel](https://www.kaggle.com/shivamb/deep-exploration-of-gun-violence-in-us).

In [0]:
## Finding the Suspect Age Groups
suspect_age_groups = {}
for i, row in guns_df.iterrows():
    suspects = []
    for k,v in row['participant_type_map'].items():
        if "suspect" in v.lower():
            suspects.append(k)
    for suspect in suspects:
        if suspect in row['participant_age_map']:
            ag = row['participant_age_map'][suspect]
            if ag not in suspect_age_groups:
                suspect_age_groups[ag] = 0 
            else:
                suspect_age_groups[ag] += 1

In [19]:
# Plotting suspects age distribution
source = pd.DataFrame({
    'a': list(suspect_age_groups.keys()),
    'b': list(suspect_age_groups.values())
})
alt.Chart(source).mark_bar(size = 80).encode(
    alt.X('a:N', title='Age Group', 
         sort=alt.EncodingSortField(
            field="b",
            op="sum",
            order="ascending"
        )),
    alt.Y('b:Q', title = 'Number of Suspects',axis=alt.Axis(format='s')),
    tooltip = ("b")
).properties(
    title='Suspects: Age Distribution',
    width=400,
    height=400
).configure_axis(
    grid=False
)

Chart({
  config: Config({
    axis: AxisConfig({
      grid: False
    })
  }),
  data:             a       b
  0  Adult 18+   151071
  1  Teen 12-17  12849 
  2  Child 0-11  577   ,
  encoding: FacetedEncoding({
    tooltip: Tooltip({
      shorthand: 'b'
    }),
    x: X({
      shorthand: 'a:N',
      sort: EncodingSortField({
        field: 'b',
        op: 'sum',
        order: 'ascending'
      }),
      title: 'Age Group'
    }),
    y: Y({
      axis: Axis({
        format: 's'
      }),
      shorthand: 'b:Q',
      title: 'Number of Suspects'
    })
  }),
  height: 400,
  mark: MarkDef({
    size: 80,
    type: 'bar'
  }),
  title: 'Suspects Age Distribution',
  width: 400
})

From the above chart we see that most incidents are carried out by adults older than 18 years but we also see a significant number of incidents involving teens aged 12-17 years.

In [0]:
## Finding the Suspect's Gender
suspect_gender = {}
for i, row in guns_df.iterrows():
    suspects = []
    for k,v in row['participant_type_map'].items():
        if "suspect" in v.lower():
            suspects.append(k)
    for suspect in suspects:
        if suspect in row['participant_gender_map']:
            g = row['participant_gender_map'][suspect]
            if g not in suspect_gender:
                suspect_gender[g] = 0 
            else:
                suspect_gender[g] += 1

In [21]:
source = pd.DataFrame({
    'a': list(suspect_gender.keys()),
    'b': list(suspect_gender.values())
})

alt.Chart(source).mark_bar(size = 80).encode(
    alt.X('a:N', title='Gender'),
    alt.Y('b:Q', title = 'Number of Suspects',axis=alt.Axis(format='s')),
    tooltip = ("b")
).properties(
    title='Suspects: Gender Distribution',
    width=400,
    height=400
).configure_axis(
    grid=False
)


Chart({
  config: Config({
    axis: AxisConfig({
      grid: False
    })
  }),
  data:         a       b
  0  Female  11745 
  1  Male    167707,
  encoding: FacetedEncoding({
    tooltip: Tooltip({
      shorthand: 'b'
    }),
    x: X({
      shorthand: 'a:N',
      title: 'Gender'
    }),
    y: Y({
      axis: Axis({
        format: 's'
      }),
      shorthand: 'b:Q',
      title: 'Number of Suspects'
    })
  }),
  height: 400,
  mark: MarkDef({
    size: 80,
    type: 'bar'
  }),
  title: 'Suspects Gender Distribution',
  width: 400
})

From the above chart we can see that most of the incidents are committed by Males.

### Analysis on Gun ownership in US by state
In the [article on Vox's website](https://www.vox.com/policy-and-politics/2017/10/2/16399418/us-gun-violence-statistics-maps-charts), chart 5 depicts the relationship between gun ownership and gun deaths across states in USA. The chart was created using 2013 data for gun ownership and gun deaths. 

Since we have more recent data (upto March 2018), we were interested in exploring this relationship with gun ownership data for a later time period. We found [2017 gun ownership by state](https://www.thoughtco.com/gun-owners-percentage-of-state-populations-3325153) as compiled by the website [HuntingMark.com](https://huntingmark.com/gun-ownership-stats/#_ftn1%20). The data on this website is taken from ATF (United States Department of Justice Bureau of Alcohol, Tobacco, Firearms and Explosives) report on [Firearm Commerce in the United States, 2017 statistics](https://www.atf.gov/resource-center/docs/undefined/firearms-commerce-united-states-annual-statistical-update-2017/download) **Exhibit 8** on **National Firearms Act Registered Weapons by State (April 2017)** which lists the number of registered guns by state.

It is important to note that the actual number of guns might be much more than the numbers depicted here since United States does not have a regulation in place for registration of all guns. But this is the most reliable, if any, data source available to us on gun ownership so we decided to use this for our analysis.



In [0]:
# importing gun ownership data by state
state_guns_owned = pd.read_csv('https://raw.githubusercontent.com/strivedi2/Gun-Violence-in-United-States/master/Statewise_Gun_Ownership2017.csv')

In [23]:
state_guns_owned.head(1)

,State,Any Other Weapon1,Destructive Device2,Machinegun3,Silencer4,Short Barreled Rifle5,Short Barreled Shotgun6,Total
0,Alabama,"1,203","78,434","26,307","48,118","5,285","2,294","161,641"


In order to normalize gun ownership by state population we also decided to add state population data for 2017 from [here](https://www.enchantedlearning.com/usa/states/population.shtml). 

We will now load state population data and create a new dataframe with total guns and population columns for each state for 2017.  

In [24]:
# load state population dataset
state_pop = pd.read_csv('https://raw.githubusercontent.com/strivedi2/Gun-Violence-in-United-States/master/StatePopulation%202017.csv')
state_pop.head(1)

,State,Population (2017)
0,California,"39,536,653"


In [0]:
# merge gun ownership and state population dataframes
state_data = state_guns_owned.merge(state_pop, how ='inner', left_on='State', right_on='State')

# drop remaining gun type columns since we wont be using them for our analysis 
state_data.drop(columns=['Any Other Weapon1','Destructive Device2','Machinegun3','Silencer4','Short Barreled Rifle5','Short Barreled Shotgun6'],inplace=True)

In [26]:
state_data.head(2)

,State,Total,Population (2017)
0,Alabama,"161,641","4,874,747"
1,Alaska,"15,824","739,795"


We will now merge the above dataframe with our original dataset to add these columns.

In [0]:
guns_df = guns_df.merge(state_data, how='left', left_on='state', right_on='State')
guns_df.drop(columns='State',inplace=True)

# rename the Total column to Total_guns
guns_df.rename(columns={'Total':'Total_guns'},inplace=True)

### Data dictionary of cleaned Dataset 1: Gun Violence Dataset on Kaggle
Column Name  | Description
--- | ---
incident_id | Incident ID
date	| Date of crime (Format: YYYY-MM-DD)
state	|State where crime was committed	
city_or_county	|City/ County of crime	
n_killed	|Number of people killed in the incident	
n_injured	|Number of people injured in the incident	
congressional_district |Congressional district id	
latitude	|Latitude of the location of the incident	
longitude	|Longitude of the location of the incident	
participant_age_group	|Age group of participant(s) (Child, Teen, Adult)	
participant_gender	|Gender of participant(s)	
participant_name	|Name of participant(s) involved in crime	
participant_relationship |Relationship of participant to other participant(s)	
participant_status	|Extent of harm done to the participant (Injured, Killed, Arrested)	
participant_type	|Type of participant (Victim, Suspect)	
state_house_district	|Voting house district	
state_senate_district	|Territorial district from which a senator to a state legislature is elected
year | Year of incident
month | Month of incident
monthday | Day of incident
weekday | Day of the week of incident (Mon, Tue, etc.)
Mass_Shooting | Incident characteristic: 1 means Mass Shooting
Officer_Involved_Shooting | Incident characteristic: 1 means Officer is involved in the shooting
Suicide | Incident characteristic: 1 means Suicide
Gun_Possession_felon | Incident characteristic: 1 means Gun possed by a felon
Accidental_Shootings | Incident characteristic: 1 means Shooting occured by Accident
participant_type_map | Map Participant type
participant_age_map | Map Participant age
participant_gender_map | Map Participant gender
Total_guns|Total guns owned in each State in 2017
Population (2017)|Population of the State in 2017

## **Dateset 2: Gun ownership by country**

We also wanted to understand the relationship between gun ownership and gun violence incidents across countries.

[Here](https://docs.google.com/spreadsheets/d/1chqUZHuY6cXYrRYkuE0uwXisGaYvr7durZHJhpLGycs/edit#gid=0) is the dataset on Gun homicide and gun ownership listed by country. More information about the dataset can be found [here](https://www.theguardian.com/news/datablog/2012/jul/22/gun-homicides-ownership-world-list)

**About the dataset:**
The world’s crime figures are collected by the UNODC (United nations office on drugs and crimes) through its annual crime survey. It has a special section of data on firearm homicides - and provides detailed information by size of population and compared to other crimes. 

**Limitations of the dataset:** <br>
1. Some key nations are missing from the data, including Russia, China and Afghanistan. But it does include the US, UK and many other developed nations. <br>
2. Also, this dataset is from 2012. We know that we will not be able to do comparison for the same time period as gun ownership dataset for United State but we are still interested in knowing if there exists any trend.



In [0]:
# We saved the data from [google sheet](https://docs.google.com/spreadsheets/d/1chqUZHuY6cXYrRYkuE0uwXisGaYvr7durZHJhpLGycs/edit#gid=0) to a csv file and uploaded it on github
# importing gun ownership data by country
gun_ownership = pd.read_csv('https://raw.githubusercontent.com/strivedi2/Gun-Violence-in-United-States/master/World_firearms.csv')

In [29]:
gun_ownership.head(2)

,Country/Territory,ISO code,Source,% of homicides by firearm,Number of homicides by firearm,"Homicide by firearm rate per 100,000 pop",Rank by rate of ownership,Average firearms per 100 people,Average total all civilian firearms
0,Albania,AL,CTS,65.9,56.0,1.76,70.0,8.6,270000.0
1,Algeria,DZ,CTS,4.8,20.0,0.06,78.0,7.6,1900000.0


In [0]:
# Removed ISO code column as we already have country information we don't need the country code for analysis
# Removed Source column as it is not relevant for our analysis
# Removed Rank by rate of ownership as rank will not be relevant for analysis
gun_ownership = gun_ownership[['Country/Territory','% of homicides by firearm',\
                               'Number of homicides by firearm','Homicide by firearm rate per 100,000 pop',\
                               'Average firearms per 100 people','Average total all civilian firearms']]

In [0]:
# Renamed few columns
gun_ownership.columns = ['Country', 'Percentage of Homicides by firearm','Number of homicide by firearm',\
                        'Homicide by firearm rate per 100,000 pop','Average firearms per 100 people','Average total all civilian firearms']

In [32]:
gun_ownership.isna().sum()

Country                                     0 
Percentage of Homicides by firearm          69
Number of homicide by firearm               69
Homicide by firearm rate per 100,000 pop    69
Average firearms per 100 people             9 
Average total all civilian firearms         9 
dtype: int64

In [0]:
# Filled the na values with 0
gun_ownership = gun_ownership.fillna(0)

In [34]:
gun_ownership.isna().sum()

Country                                     0
Percentage of Homicides by firearm          0
Number of homicide by firearm               0
Homicide by firearm rate per 100,000 pop    0
Average firearms per 100 people             0
Average total all civilian firearms         0
dtype: int64

In [0]:
# Creating a dataframe of top 20 countries with the highest average firearms per 100 people by Country
top20 = gun_ownership[['Country','Average firearms per 100 people']].sort_values('Average firearms per 100 people', ascending = False).nlargest(20, 'Average firearms per 100 people')

In [36]:
alt.Chart(top20).mark_bar().encode(
    alt.X('Country', title = 'Country', 
         sort=alt.EncodingSortField(
            field="Average firearms per 100 people",
            #op="sum",
            order="descending"
        )),
    alt.Y('Average firearms per 100 people', title = 'Average firearms per 100 people')
).properties(
    title='Average firearms per 100 people for top 20 countries',
    width=650,
    height=400
).configure_axis(
    grid=False
)

Chart({
  config: Config({
    axis: AxisConfig({
      grid: False
    })
  }),
  data:            Country  Average firearms per 100 people
  176  United States  88.8                           
  182  Yemen          54.8                           
  162  Switzerland    45.7                           
  58   Finland        45.3                           
  147  Serbia         37.8                           
  43   Cyprus         36.4                           
  144  Saudi Arabia   35.0                           
  79   Iraq           34.2                           
  177  Uruguay        31.8                           
  161  Sweden         31.6                           
  127  Norway         31.3                           
  59   France         31.2                           
  29   Canada         30.8                           
  7    Austria        30.4                           
  75   Iceland        30.3                           
  63   Germany        30.3                           
  128  Oman           25.4                           
  91   Kuwait         24.8                           
  10   Bahrain        24.8                           
  102  Macedonia      24.1                           ,
  encoding: FacetedEncoding({
    x: X({
      shorthand: 'Country',
      sort: EncodingSortField({
        field: 'Average firearms per 100 people',
        order: 'descending'
      }),
      title: 'Country'
    }),
    y: Y({
      shorthand: 'Average firearms per 100 people',
      title: 'Average firearms per 100 people'
    })
  }),
  height: 400,
  mark: 'bar',
  title: 'Average firearms per 100 people for top 20 countries',
  width: 650
})

From the above chart we can see that United states is the highest in terms of average firearms per 100 people.

### Data dictionary of Dataset 2: Gun ownership by Country
Column Name  | Description
--- | ---
Country | Country Name
Percentage of Homicides by firearm | Percentage of Homicides by firearm
Number of homicide by firearm | Number of homicides by firearm
Homicide by firearm rate per 100,000 pop | Homicides by firearm rate per 100,000 population
Average firearms per 100 people | Average firearms per 100 people
Average total all civilian firearms | Average total of civilians who owned firearms

## Road map

We want to explore the following trends/relationships and also replicate/improve the existing charts on the [Vox article](https://www.vox.com/policy-and-politics/2017/10/2/16399418/us-gun-violence-statistics-maps-charts)

1. **Relationship between gun ownership and gun violence:** We would like to understand if there exists any relationship between gun ownership and gun violence incidents across US states, and also see if similar relationships can be found with the data available for countries. 
2. **Gun violence incident characteristics:** We would like to understand the trend of Mass shootings, Suicides and Officers involved shootings across US. 
3. **Understanding suspect profiles with respect to gun violence incidents:** We would like to explore gun violence incidents and their characterstics with respect to suspect age - teens (age 12- 17) and adults ( aged 18 years and olders)

We are also interested in exploring gun ownership and gun violence incidents with respect to **gun laws** across states in US in later versions.